https://towardsdatascience.com/knowing-these-you-can-cover-99-of-file-operations-in-python-84725d82c2df

# Нативные способы создать, открыть, прочитать и закрыть файлы

In [6]:
import os
print(os.getcwd())  # посмотреть рабочую директорию
os.chdir("./Files")  # сменить рабочую директорию
print(os.getcwd())  # посмотреть снова рабочую директорию

/Users/iakubovskii/Репетитор/Development/6. Работа с данными
/Users/iakubovskii/Репетитор/Development/6. Работа с данными/Files


In [7]:
file = open("file1.txt","w+")  # w+ означает запись
file.write("a new line")  # запишем текст
file.close()  # закроем файл

Основные методы работы с файлами:

r	- чтение (по умолчанию)
r+	- и чтение и запись (находимся в начале самого файла)
w	- запись (обрезает файл, если такой файл уже существует)
w+	- открыть для записи и чтения (обрезает файл, если такой файл уже существует)
a	- открываем для добавления новых данных (добавляет данные, если файл существует; начинает с конца файла)

In [8]:
# Открываем файлы правильно
with open("file1.txt", "a") as file1:
    file1.write("\nthe second line")

Однако, открытие при помощи контекстного менеджера сопряжено с потерей производительности.
Все дело в том, что мы каждый раз закрываем и открываем файл.
С другой стороны, это сильно помогает в случае каких-то непредвиденных ошибок.
Наши данные будут надежно сохранены в файле.

In [9]:
def _write_to_file(file, line):
    with open(file, "a") as f:
        f.write(line)

def _valid_records():
    for i in range(100000):
        if i % 2 == 0:
            yield i

def use_context_manager_2(file):
    for line in _valid_records():
        _write_to_file(file, str(line))

def use_context_manager_1(file):
    with open(file, "a") as f:
        for line in _valid_records():
            f.write(str(line))

def use_close_method(file):
    f = open(file, "a")
    for line in _valid_records():
        f.write(str(line))
    f.close()

%time use_close_method("test.txt")
%time use_context_manager_1("test.txt")
%time use_context_manager_2("test.txt")

CPU times: user 54.6 ms, sys: 9.61 ms, total: 64.2 ms
Wall time: 99.1 ms
CPU times: user 41.5 ms, sys: 4.42 ms, total: 45.9 ms
Wall time: 62.6 ms
CPU times: user 2.61 s, sys: 4.72 s, total: 7.33 s
Wall time: 12.9 s


Существует 3 способа прочитать данные в файле:
- read()
- readline()
- readlines()

По умолчанию `read(size=-1)` возвращает все содержимое файла. Если файл больше памяти, необязательный параметр size может помочь вам ограничить размер возвращаемых символов (текстовый режим) или байтов (двоичный режим).

`readline(size=-1)` возвращает всю строку, включая символ \n в конце. Если size больше 0, то возвращается максимальное количество символов из строки.

`readlines(hint=-1)` возвращает все строки файла в виде списка. Необязательный параметр hint означает, что если количество возвращаемых символов превысит hint, то больше строк возвращено не будет.

In [10]:
with open('file1.txt', 'r') as reader:
    line = reader.readline()
    while line != "":
        line = reader.readline()
        print(line)

the second line



Для записи существует 2 метода `write()` и `writelines()`.
Как следует из названия, `write()` предназначен для записи строки, а `writelines()` - для записи списка строк. Ответственность за добавление \n в конце лежит на разработчике.

In [11]:
with open("test1.txt", "w+") as f:
    f.write("hi\n")
    f.writelines(["this is a line\n", "this is another line\n"])

Для .csv и .json форматов существуют специальные модули

In [12]:
import csv
import json

with open("cities.csv", "w+") as file:
    writer = csv.DictWriter(file, fieldnames=["city", "country"])
    writer.writeheader()
    writer.writerow({"city": "Amsterdam", "country": "Netherlands"})
    writer.writerows(
        [
            {"city": "Berlin", "country": "Germany"},
            {"city": "Shanghai", "country": "China"},
        ]
    )

In [15]:
# Добавим данных в csv
with open("cities.csv", "a") as file:
    writer = csv.writer(file)
    writer.writerow(['Moscow', 'Russia'])

In [16]:
with open('cities.csv', 'r') as file:
    print(file.read())

city,country
Amsterdam,Netherlands
Berlin,Germany
Shanghai,China
Moscow,Russia
Moscow,Russia,Capital



In [17]:
with open("cities.json", "w+") as file:
    json.dump({"city": "Amsterdam", "country": "Netherlands"}, file)

In [18]:
with open("cities.json", "r") as file:
    our_dict = json.load(file)
our_dict

{'city': 'Amsterdam', 'country': 'Netherlands'}

# Работа с файлами из Python

In [19]:
print(os.stat("cities.csv"))

os.stat_result(st_mode=33188, st_ino=4395533865, st_dev=16777220, st_nlink=1, st_uid=501, st_gid=20, st_size=107, st_atime=1643916535, st_mtime=1643916533, st_ctime=1643916533)


In [31]:
from datetime import datetime
datetime.fromtimestamp(1643916535)

datetime.datetime(2022, 2, 3, 22, 28, 55)

- st_size - размер файла в байтах
- st_atime - время последнего доступа
- st_mtime - время последней модификации

In [34]:
print(os.path.getatime("cities.csv"))
print(os.path.getctime("cities.csv"))
print(os.path.getmtime("cities.csv"))
print(os.path.getsize("cities.csv"))

1643916535.9090462
1643916533.8795009
1643916533.8795009
107


In [35]:
# Скопировать файл
import shutil
shutil.copy("file1.txt", "file1_copy.txt")

'file1_copy.txt'

Можно также обращаться через Python к командной строке через модуль `os`.
Заметим, что рабочая директория в этом случае остается как и в Python.

In [33]:
# Копировать
os.system("cp file1.txt file1_copy_copy.txt")

# Переименовать / сменить расположение
os.system("mv cities.csv cities_new.csv")
os.rename("test1.txt", "test11.txt")

# Удалить
os.system("rm file1_copy_copy.txt")

# Поиск файлов

In [36]:
import glob
print(glob.glob("*.csv"))
print(glob.glob("**/*.csv",recursive=True))

['cities.csv', 'cities_new.csv']
['cities.csv', 'cities_new.csv']


In [37]:
for file in os.listdir("."):
    if file.endswith(".csv"):
        print(file)

for root, dirs, files in os.walk("."):
    for file in files:
        if file.endswith(".csv"):
            print(file)

.csv
cities.csv
cities_new.csv
.csv
cities.csv
cities_new.csv


## pathlib

In [38]:
from pathlib import Path

p = Path(".")
for name in p.glob("**/*.csv"): # recursive
    print(name)

.csv
cities.csv
cities_new.csv


In [38]:
import pathlib
print(os.path.abspath("1.txt"))  # absolute
print(os.path.relpath("1.txt"))  # relative

print(pathlib.Path("1.txt").absolute())  # absolute
print(pathlib.Path("1.txt"))  # relative

/Users/iakubovskii/Репетитор/Development/6. Работа с данными/Files/1.txt
1.txt
/Users/iakubovskii/Репетитор/Development/6. Работа с данными/Files/1.txt
1.txt


In [40]:
# Присоединяем к пути еще папки
print(os.path.join("/home", "file.txt"))
print(pathlib.Path("/home") / "file.txt")

/home/file.txt
/home/file.txt


In [42]:
# relative path
print(os.path.dirname("Files/cities_new.csv"))
# source
print(pathlib.Path("Files/cities_new.csv").parent)
# source

# absolute path
print(pathlib.Path("Files/cities_new.csv").resolve().parent)
print(os.path.dirname(os.path.abspath("Files/cities_new.csv")))

Files
Files
/Users/iakubovskii/Репетитор/Development/6. Работа с данными/Files/Files
/Users/iakubovskii/Репетитор/Development/6. Работа с данными/Files/Files


Как говорится в документации Python, `pathlib` - это более объектно-ориентированное решение, чем `os`. Он представляет каждый путь к файлу как объект, а не как строку. Это дает много преимуществ разработчикам, например, упрощает объединение нескольких путей, является более последовательным на разных операционных системах, методы доступны непосредственно из объекта.

# pandas

Это самый легкий способ записать и прочитать файлы любого формата в Python.
Однако и самый затратный по памяти. Но если данных немного, то можно на это забить и радоваться удобству pandas.

In [41]:
import pandas as pd
df = pd.read_csv("cities_new.csv")
df.append(['Kiev', 'Ukraine'])
df.to_csv("cities_new.csv", index=False, mode='a', header=True)

In [42]:
pd.read_csv("cities_new.csv")

,city,country
0,Amsterdam,Netherlands
1,Berlin,Germany
2,Shanghai,China
3,Moscow,Russia
4,city,country
5,Amsterdam,Netherlands
6,Berlin,Germany
7,Shanghai,China
8,Moscow,Russia
